# Biclustering documents with the Spectral Co-clustering algorithm
- 20개의 뉴스 그룹 데이터 셋에 대한 Spectrum Co-clustering 알고리즘을 보인다.
    - `comp.os.ms-window.misc` 카테고리는 데이터만 포함된 게시물이 많기 때문에 제외된다.
- TF-IDF 벡터화된 게시물은 단어 빈도 행렬을 형성한 다음 Spectrum Co-clustering 알고리즘을 사용하여 바이클러스터링 된다.
    - 결과적인 문서-단어 바이클러스터는 해당 하위 집합 문서에서 더 자주 사용되는 하위 집합 단어를 나타낸다.
- 베스트 바이클러스터의 경우, 가장 일반적인 문서 카테고리와 10개의 가장 중요한 단어가 프린트된다.
    - 베스트 바이클러스터는 정규화된 cut에 의해 결정된다.
    - 중요한 단어는 바이클러스터 내부와 외부의 합계를 비교하여 결정된다.
- 비교를 위해 `MiniBatchKmeans`를 사용하여 문서를 클러스터링 한다.
    - 바이클러스터에서 파생된 문서 클러스터는 `MiniBatchKmeans`에서 찾은 클러스터보다 더 나은 V-measure를 달성한다.

- Biclustering(양방향 클러스터링): 데이터를 동시에 행과 열 양쪽에서 클러스터로 그룹화하는 기술
    - 유전자 발현 데이터
    - 이미지 분석
    - 텍스트 마이닝
- Spectral Co-clustering algorithm: 바이클러스터링 알고리즘 중 하나, 행렬의 구조적 패턴을 파악하고자 할 때 사용
    - 데이터를 특잇값 분해하여 잠재적인 패턴 추출
    - 추출된 특잇값을 기반으로 그래프 생성
    - 그래프 분할 알고리즘을 사용하여 행과 열을 클러스터로 그룹화

In [2]:
import operator
from collections import defaultdict
from time import time
from typing import Callable

import numpy as np

from sklearn.cluster import MiniBatchKMeans, SpectralCoclustering
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.cluster import v_measure_score

### 토큰 리스트를 받아와서 해당 리스트의 각 토큰이 숫자로 시작하면 그 토큰을 `#NUMBER`로 대체하는 함수인 `number_normalizer`를 정의한다.
- 텍스트 데이터를 처리할 때, 숫자로 시작하는 단어들은 불필요한 노이즈로 작용할 수 있다.
- 숫자로 시작하는 토큰들을 `#NUMBER`로 대체하여 차원 축소의 효과를 낸다.
    - ['123abc', 'apple', '45xyz'] $->$ ['#NUMBER', 'apple', '#NUMBER']

In [3]:
def number_normalizer(tokens):
    """
    모든 숫자 토큰을 placeholder에 매핑한다.
    많은 application에서 숫자로 시작하는 토근은 직접적으로 유용하지는 않지만 토큰이 존재한다는 사실이 관련될 수는 있다.
    이러한 형태의 차원 감소를 적용함으로 일부 방법은 더 나은 성능을 발휘할 수 있다.
    """
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)

### 20개의 다른 주제를 가진 뉴스 그룹 데이터를 처리하고, 특정 벡터화 및 클러스터링 기법을 적용하여 문서를 그룹화하는 과정
- `NumberNormalizingVectorizer`
    - `TfidfVectorizer`를 상속받아 만들어진다.
    - `build_tokenizer` 메서드를 오버라이드하여, 기존의 토크나이저에 추가적인 숫자 정규화 기능을 포함한다.
- 데이터 로딩 및 전처리:
    - 뉴스 그룹 데이터를 `fetch_20newsgroups`함수를 사용해 불러온다.
    - `NumberNormalizingVectorizer`를 사용하여 TF-IDF 벡터화를 수행한다.
        - stop words를 제거하고, 최소 문서 빈도(`min_df`)가 5인 단어만 사용한다.
- Co-clustering, Kmeans clustering
    - `SpectralCoclustering`과 `MiniBatchKMeans`를 사용하여 각각 코 클러스터링과 K-평균 클러스터링을 수행한다.
- 성능 평가
    - `v_measure_score`함수를 사용해 클러스터링 결과의 성능을 측정한다.
        - V-measure는 정밀도(Precision)와 재현율(Recall)의 조화 평균으로 계산되는 클러스터링 지표이다. 
- 출력
    - 클러스터링 과정과 결과의 V-measure 스코어가 출력된다.

In [4]:
class NumberNormalizingVectorizer(TfidfVectorizer):
    def build_tokenizer(self):
        tokenize = super().build_tokenizer()
        return lambda doc: list(number_normalizer(tokenize(doc)))
    
# comp.os.ms-windows.misc 제외
categories = [
    "alt.atheism",
    "comp.graphics",
    "comp.sys.ibm.pc.hardware",
    "comp.sys.mac.hardware",
    "comp.windows.x",
    "misc.forsale",
    "rec.autos",
    "rec.motorcycles",
    "rec.sport.baseball",
    "rec.sport.hockey",
    "sci.crypt",
    "sci.electronics",
    "sci.med",
    "sci.space",
    "soc.religion.christian",
    "talk.politics.mideast",
    "talk.politics.misc",
    "talk.religion.misc",
]
newsgroups = fetch_20newsgroups(categories=categories)
y_true = newsgroups.target

vectorizer = NumberNormalizingVectorizer(stop_words="english", min_df=5)
cocluster = SpectralCoclustering(
    n_clusters=len(categories), svd_method="arpack", random_state=0
)
kmeans = MiniBatchKMeans(
    n_clusters=len(categories), batch_size=20000, random_state=0, n_init=3
)

print("Vectorizing...")
X = vectorizer.fit_transform(newsgroups.data)

print("Coclustering...")
start_time = time()
cocluster.fit(X)
y_cocluster = cocluster.row_labels_
print(
    "Done in {:.2f}s. V-measure: {:.4f}".format(time() - start_time, v_measure_score(y_cocluster, y_true))
)

print("MiniBatchKmeans...")
start_time = time()
y_kemans = kmeans.fit_predict(X)
print(
     "Done in {:.2f}s. V-measure: {:.4f}".format(time() - start_time, v_measure_score(y_kemans, y_true))
)

Vectorizing...
Coclustering...
Done in 0.54s. V-measure: 0.4487
MiniBatchKmeans...
Done in 1.95s. V-measure: 0.2690


### Co-clustering의 결과에서 바이클러스터에 대한 Normalized Cut(Ncut)을 꼐산하는 함수 정의
- Normalized Cut은 클러스터 내의 가중치를 클러스터 외의 가중치로 나눈 값으로 클러스터링의 품질을 측정하는 지표 중 하나이다.
- `bicluster_ncut(i)`: `i`는 바이클러스터의 인덱스를 나타낸다.
    - `cocluster.get_indices(i)`: 바이클러스터 `i`의 행과 열 인덱스를 가져온다.
    - `np.logical_not(cocluster.rows_[i])`: 행과 열에 대한 complement 인덱스(주어진 집합에 속하지 않는 모든 원소들로 이루어진 집합)를 계산한다.
    - `X[rows][:, cols].sum()`: 바이클러스터 내의 가중치 합을 계산한다.
    - `X[row_complement][:, cols].sum() + X[rows][:, col_complement].sum()`: 바이클러스터 외의 가중치 합을 계산한다.

In [5]:
feature_names = vectorizer.get_feature_names_out()
document_names = list(newsgroups.target_names[i] for i in newsgroups.target)

def bicluster_ncut(i):
    rows, cols = cocluster.get_indices(i)
    if not (np.any(rows) and np.any(cols)):
        import sys

        return sys.float_info.max
    row_complement = np.nonzero(np.logical_not(cocluster.rows_[i]))[0]
    col_complement = np.nonzero(np.logical_not(cocluster.columns_[i]))[0]
    # X[rows[:, np.newaxis], cols].sum()이 scipy <= 0.16  
    weight = X[rows][:, cols].sum()
    cut = X[row_complement][:, cols].sum() + X[rows][:, col_complement].sum()
    return cut / weight

### 바이클러스터의 품질을 측정한 결과를 분석하고, 가장 좋은 바이클러스터를 선택해 해당 바이클러스터에 속하는 문서와 단어들, 카테고리 및 중요한 단어들을 출력
- `most_common(d)` 함수
    - `defaultdict(int)`형태의 딕셔너리를 받아 해당 딕셔너리를 내림차순으로 정렬하여 반환한다.
    - 빈도수가 높은 항목을 찾을 때 사용된다.
- `bicluster_ncuts` 계산
    - 각 바이클러스터에 대한 Normalized Cut 값을 계산하여 리스트에 저장한다.
- 가장 좋은 바이클러스터 선택:
    - `np.argsort(bicluster_ncuts)[:5]`을 사용해 Normalized Cut 값이 잔은 상위 5개의 바이클러스터를 선택한다.
- 선택된 바이클러스터에 대한 분석 및 출력:
    - 선택된 바이클러스터에 대해 해당하는 문서와 단어들을 가져와서 분석한다.
    - 바이클러스터에 속한 문서의 카테고리 분포를 계산하고 출력한다.
    - 바이클러스터에 속한 단어들 중에서 중요한 단어들을 계산하고 출력한다.

In [7]:
def most_common(d):
    """
    가장 높은 값을 가진 defaultdict(int)의 항목이다.
    Python >= 2.7에서 Counter.most_common과 비슷하다.
    """    
    return sorted(d.items(), key=operator.itemgetter(1), reverse=True)

bicluster_ncuts = list(bicluster_ncut(i) for i in range(len(newsgroups.target_names)))
best_idx = np.argsort(bicluster_ncuts)[:5]

print()
print("Best biclusters:")
print("----------------")
for idx, cluster in enumerate(best_idx):
    n_rows, n_cols = cocluster.get_shape(cluster)
    cluster_docs, cluster_words = cocluster.get_indices(cluster)
    if not len(cluster_docs) or not len(cluster_words):
        continue

    # categories
    counter = defaultdict(int)
    for i in cluster_docs:
        counter[document_names[i]] += 1
    cat_string = ", ".join(
        "{:.0f}% {}".format(float(c) / n_rows * 100, name)
        for name, c in most_common(counter)[:3]
    )

    # words
    out_of_cluster_docs = cocluster.row_labels_ != cluster
    out_of_cluster_docs = np.where(out_of_cluster_docs)[0]
    word_col = X[:, cluster_words]
    word_scores = np.array(
        word_col[cluster_docs, :].sum(axis=0)
        - word_col[out_of_cluster_docs, :].sum(axis=0)
    )
    word_scores = word_scores.ravel()
    important_words = list(
        feature_names[cluster_words[i]] for i in word_scores.argsort()[:-11:-1]
    )

    print("bicluster {} : {} documents, {} words".format(idx, n_rows, n_cols))
    print("categories   : {}".format(cat_string))
    print("words        : {}\n".format(", ".join(important_words)))


Best biclusters:
----------------
bicluster 0 : 8 documents, 6 words
categories   : 100% talk.politics.mideast
words        : cosmo, angmar, alfalfa, alphalpha, proline, benson

bicluster 1 : 1092 documents, 2929 words
categories   : 30% talk.politics.mideast, 27% soc.religion.christian, 26% alt.atheism
words        : god, jesus, christians, atheists, morality, kent, sin, belief, objective, resurrection

bicluster 2 : 2225 documents, 2862 words
categories   : 18% comp.sys.mac.hardware, 16% comp.sys.ibm.pc.hardware, 16% sci.electronics
words        : voltage, shipping, receiver, circuit, compression, digital, processing, scope, baalke, package

bicluster 3 : 1505 documents, 3837 words
categories   : 24% talk.politics.misc, 18% sci.med, 17% soc.religion.christian
words        : geb, banks, gordon, drugs, kaldis, dyer, br, surrender, noring, n3jxp

bicluster 4 : 1722 documents, 2666 words
categories   : 27% rec.motorcycles, 23% rec.autos, 13% misc.forsale
words        : bike, car, dod, r